In [33]:
import numpy as np
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.algorithms import NumPyEigensolver
from numpy import linalg as LA
from scipy.linalg import expm, norm

In [34]:
def is_unitary(m):
    return norm(np.eye(len(m)) - m.dot(m.T.conj()))
def is_hermitian(m):
    return norm(m-m.T.conj())

In [192]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])

# comment this out when stuff actually works
bases = ['X', 'Y', 'Z'] # these are all commuting
coefficients = [1]*len(bases)
hamiltonian = np.zeros((2**len(bases[0]), 2**len(bases[0])), dtype=np.cdouble)
for c, lbl in zip(coefficients, bases):
    op = Operator(Pauli(label=lbl))
    hamiltonian+=c*op.data

In [93]:
hamiltonian

array([[0. +0.j , 0. +0.j , 0. -0.2j, 0.6-0.4j],
       [0. +0.j , 0. +0.j , 0.6+0.4j, 0. +0.2j],
       [0. +0.2j, 0.6-0.4j, 0. +0.j , 0. +0.j ],
       [0.6+0.4j, 0. -0.2j, 0. +0.j , 0. +0.j ]])

In [94]:
is_hermitian(hamiltonian)

0.0

In [95]:
def get_circuit(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    for i in range(n):
        for c, b in zip(coefficients, bases):
            # implement e^{-i*c*b*t/n}
            b = b[::-1]
            q = []
            qc.barrier()
            for i, op in enumerate(b):
                if (op == 'X'):
                    qc.h(i)
                    q.append(i)
                elif (op == 'Z'):
                    q.append(i)
                elif (op == 'Y'):
                    qc.sdg(i)
                    qc.h(i)
                    q.append(i)
                elif (op == 'I'):
                    continue
            qc.barrier()
            if (len(q) > 0):
                if (len(q) > 1):
                    for i in range(len(q)-1):
                        qc.cx(q[i], q[-1])
                qc.rz(2*c*t/n, q[-1])
                if (len(q) > 1):
                    for i in reversed(range(len(q)-1)):
                        qc.cx(q[i], q[-1])
                qc.barrier()
                for i, op in enumerate(b):
                    if (op == 'X'):
                        qc.h(i)
                    elif (op == 'Z'):
                        continue
                    elif (op == 'Y'):
                        qc.h(i)
                        qc.s(i)
                    elif (op == 'I'):
                        continue
            qc.barrier()
    return qc

def get_unitaries(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

In [96]:
get_circuit(n=1).draw()

░              ░                       ░            ░  ░ ┌─────┐┌───┐ ░ »
q_0: ─░──────────────░───■───────────────■───░────────────░──░─┤ Sdg ├┤ H ├─░─»
      ░ ┌─────┐┌───┐ ░ ┌─┴─┐┌─────────┐┌─┴─┐ ░ ┌───┐┌───┐ ░  ░ └┬───┬┘└───┘ ░ »
q_1: ─░─┤ Sdg ├┤ H ├─░─┤ X ├┤ Rz(0.4) ├┤ X ├─░─┤ H ├┤ S ├─░──░──┤ H ├───────░─»
      ░ └─────┘└───┘ ░ └───┘└─────────┘└───┘ ░ └───┘└───┘ ░  ░  └───┘       ░ »
«                           ░ ┌───┐┌───┐ ░  ░ ┌───┐ ░                       ░ »
«q_0: ──■───────────────■───░─┤ H ├┤ S ├─░──░─┤ H ├─░───■───────────────■───░─»
«     ┌─┴─┐┌─────────┐┌─┴─┐ ░ ├───┤└───┘ ░  ░ ├───┤ ░ ┌─┴─┐┌─────────┐┌─┴─┐ ░ »
«q_1: ┤ X ├┤ Rz(0.8) ├┤ X ├─░─┤ H ├──────░──░─┤ H ├─░─┤ X ├┤ Rz(1.2) ├┤ X ├─░─»
«     └───┘└─────────┘└───┘ ░ └───┘      ░  ░ └───┘ ░ └───┘└─────────┘└───┘ ░ »
«     ┌───┐ ░ 
«q_0: ┤ H ├─░─
«     ├───┤ ░ 
«q_1: ┤ H ├─░─
«     └───┘ ░

In [97]:
e_ith, U_circuit = get_unitaries(n=100)

In [98]:
def get_error(e_ith, U_circuit):
    diff = U_circuit.data - e_ith
    # measure the operator's accuracy
    w, v = LA.eig(diff)

    return np.absolute(max(w))

get_error(e_ith, U_circuit)

0.002197348302606446

In [99]:
np.around(e_ith, decimals=3)

array([[ 0.736-0.j   ,  0.073+0.109j, -0.149+0.j   , -0.359-0.538j],
       [ 0.073-0.109j,  0.736-0.j   ,  0.359-0.538j,  0.149-0.j   ],
       [ 0.149-0.j   , -0.359-0.538j,  0.736-0.j   , -0.073-0.109j],
       [ 0.359-0.538j, -0.149+0.j   , -0.073+0.109j,  0.736-0.j   ]])

In [100]:
np.around(U_circuit, decimals=3)

array([[ 0.736-0.002j,  0.073+0.109j, -0.149+0.j   , -0.359-0.538j],
       [ 0.073-0.109j,  0.736+0.002j,  0.359-0.538j,  0.149+0.j   ],
       [ 0.149-0.j   , -0.359-0.538j,  0.736-0.002j, -0.073-0.109j],
       [ 0.359-0.538j, -0.149-0.j   , -0.073+0.109j,  0.736+0.002j]])

In [91]:
hamiltonian

array([[0.+0.j, 0.+0.j, 0.-1.j, 1.-1.j],
       [0.+0.j, 0.+0.j, 1.+1.j, 0.+1.j],
       [0.+1.j, 1.-1.j, 0.+0.j, 0.+0.j],
       [1.+1.j, 0.-1.j, 0.+0.j, 0.+0.j]])

In [ ]:
e_ith-U_circuit.data

In [ ]:
np.unique(e_ith4-U_circuit4.data), np.unique(e_ith4-U_circuit4.data)

10 qubits -> 2^10 states -> 2^20 entries ($\approx$ 1,000,000, which is quite reasonable)

In [193]:
# Higher order trotter formulation.
def get_circuit_2(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    forwardOp = list(zip(coefficients, bases))
    reverseOp = list(zip(reversed(coefficients), reversed(bases)))
    
    for i in range(n):
        for c2, b2 in forwardOp:
            print(f"{b2} -> ")
            # implement e^{-i*c*b*t/n}
            b2 = b2[::-1]
            q = []
            qc.barrier()
            for i, op in enumerate(b2):
                if (op == 'X'):
                    qc.h(i)
                    q.append(i)
                elif (op == 'Z'):
                    q.append(i)
                elif (op == 'Y'):
                    qc.sdg(i)
                    qc.h(i)
                    q.append(i)
                elif (op == 'I'):
                    continue
            qc.barrier()
            if (len(q) > 0):
                if (len(q) > 1):
                    for i in range(len(q)-1):
                        qc.cx(q[i], q[-1])
                # half the regular angle since we apply this twice.
                qc.rz(c2*t/n, q[-1])
                if (len(q) > 1):
                    for i in reversed(range(len(q)-1)):
                        qc.cx(q[i], q[-1])
                qc.barrier()
                for i, op in enumerate(b2):
                    if (op == 'X'):
                        qc.h(i)
                    elif (op == 'Z'):
                        continue
                    elif (op == 'Y'):
                        qc.h(i)
                        qc.s(i)
                    elif (op == 'I'):
                        continue
            qc.barrier()
            for c1, b1 in reverseOp:
                # implement e^{-i*c*b*t/n}
                print(b1)
                b1 = b1[::-1]
                q = []
                qc.barrier()
                for i, op in enumerate(b1):
                    if (op == 'X'):
                        qc.h(i)
                        q.append(i)
                    elif (op == 'Z'):
                        q.append(i)
                    elif (op == 'Y'):
                        qc.sdg(i)
                        qc.h(i)
                        q.append(i)
                    elif (op == 'I'):
                        continue
                qc.barrier()
                if (len(q) > 0):
                    if (len(q) > 1):
                        for i in range(len(q)-1):
                            qc.cx(q[i], q[-1])
                    # half the regular angle since we apply this twice.
                    qc.rz(c1*t/n, q[-1])
                    if (len(q) > 1):
                        for i in reversed(range(len(q)-1)):
                            qc.cx(q[i], q[-1])
                    qc.barrier()
                    for i, op in enumerate(b1):
                        if (op == 'X'):
                            qc.h(i)
                        elif (op == 'Z'):
                            continue
                        elif (op == 'Y'):
                            qc.h(i)
                            qc.s(i)
                        elif (op == 'I'):
                            continue
                qc.barrier()
    return qc

def get_unitaries_2(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit_2(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

In [194]:
get_circuit_2(n=1).draw()

X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X


░ ┌───┐ ░ ┌───────┐ ░ ┌───┐ ░  ░  ░ ┌───────┐ ░  ░  ░ ┌─────┐┌───┐ ░ »
q: ─░─┤ H ├─░─┤ Rz(1) ├─░─┤ H ├─░──░──░─┤ Rz(1) ├─░──░──░─┤ Sdg ├┤ H ├─░─»
    ░ └───┘ ░ └───────┘ ░ └───┘ ░  ░  ░ └───────┘ ░  ░  ░ └─────┘└───┘ ░ »
«   ┌───────┐ ░ ┌───┐┌───┐ ░  ░ ┌───┐ ░ ┌───────┐ ░ ┌───┐ ░  ░ ┌─────┐┌───┐ ░ »
«q: ┤ Rz(1) ├─░─┤ H ├┤ S ├─░──░─┤ H ├─░─┤ Rz(1) ├─░─┤ H ├─░──░─┤ Sdg ├┤ H ├─░─»
«   └───────┘ ░ └───┘└───┘ ░  ░ └───┘ ░ └───────┘ ░ └───┘ ░  ░ └─────┘└───┘ ░ »
«   ┌───────┐ ░ ┌───┐┌───┐ ░  ░  ░ ┌───────┐ ░  ░  ░ ┌─────┐┌───┐ ░ ┌───────┐»
«q: ┤ Rz(1) ├─░─┤ H ├┤ S ├─░──░──░─┤ Rz(1) ├─░──░──░─┤ Sdg ├┤ H ├─░─┤ Rz(1) ├»
«   └───────┘ ░ └───┘└───┘ ░  ░  ░ └───────┘ ░  ░  ░ └─────┘└───┘ ░ └───────┘»
«    ░ ┌───┐┌───┐ ░  ░ ┌───┐ ░ ┌───────┐ ░ ┌───┐ ░  ░  ░ ┌───────┐ ░  ░  ░  ░ »
«q: ─░─┤ H ├┤ S ├─░──░─┤ H ├─░─┤ Rz(1) ├─░─┤ H ├─░──░──░─┤ Rz(1) ├─░──░──░──░─»
«    ░ └───┘└───┘ ░  ░ └───┘ ░ └───────┘ ░ └───┘ ░  ░  ░ └───────┘ ░  ░  ░  ░ »
«   ┌───────┐ ░  ░  ░ ┌─────┐┌───┐ ░ ┌───────┐ ░ ┌───┐┌───┐ ░  ░ ┌───┐ ░ »
«q: ┤ Rz(1) ├─░──░──░─┤ Sdg ├┤ H ├─░─┤ Rz(1) ├─░─┤ H ├┤ S ├─░──░─┤ H ├─░─»
«   └───────┘ ░  ░  ░ └─────┘└───┘ ░ └───────┘ ░ └───┘└───┘ ░  ░ └───┘ ░ »
«   ┌───────┐ ░ ┌───┐ ░ 
«q: ┤ Rz(1) ├─░─┤ H ├─░─
«   └───────┘ ░ └───┘ ░

In [195]:
e_ith, U_circuit = get_unitaries_2(n=100)

X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z -> 
Z
Y
X
X -> 
Z
Y
X
Y -> 
Z
Y
X
Z ->

In [196]:
get_error(e_ith, U_circuit)

1.5253524870061206

In [197]:
np.around(e_ith, decimals=3)

array([[-0.161-0.57j, -0.57 -0.57j],
       [ 0.57 -0.57j, -0.161+0.57j]])

In [198]:
np.around(U_circuit, decimals=3)

array([[-0.948+0.184j,  0.185+0.184j],
       [-0.185+0.184j, -0.948-0.184j]])

In [199]:
w, v = LA.eig(hamiltonian)

In [200]:
w

array([ 1.73205081+9.06493304e-17j, -1.73205081-9.06493304e-17j])